# Programming Assignment: Social Network Database Fix

## Welcome!

You've reached a practical part of our course, **Generative AI for Software Development - AI-Powered Software and System Design**. This task is all about diving into a database designed for managing a social network. It's a hands-on challenge to test your skills and see how well you can work with AI tools, like a Language Learning Model (LLM).

## Your Mission

Your job is to fix a database code that's not working right. It's supposed to handle a social network with two main parts: a `Person` table and a `Club` table. There's a mistake in the code, and it's up to you to find and correct it.

### Tasks:

1. **Fix the Database:** Hunt down the mistake messing up the database and correct it.
2. **Write Three Functions:** Use the LLM to help you craft three functions. They should:
   - List all members of a specific club.
   - List all friends of a specific person.
   - List all people who consider a specific person their friend.

### Working with the LLM:

- **Ask the LLM for Help:** Use the LLM to guide you through fixing the database and creating your functions.
- **Use Its Advice Wisely:** Remember, the LLM's advice might not always be spot-on. It's up to you to decide what's useful.

## What to Submit:

Please submit this Jupyter notebook containing:
- The corrected database code.
- The three functions you've written.

## How We'll Grade It:

- **Database Accuracy:** We'll check if you've successfully identified and fixed the issue in the database.
- **Functionality of Your Functions:** We'll see if each of your functions is doing exactly what it's supposed to do.

## Tips for Success:

- **Be Clear with Your Questions:** The clearer your questions to the LLM, the better help you'll get.
- **Test the Advice:** Always test out the LLM’s suggestions to ensure they work as expected.
- **Hints**: If you struggle with this assignment, you can check some hints we left to you in the bottom of the assignment!

### Necessary imports

In [1]:
import os
import numpy as np
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table
from sqlalchemy.orm import declarative_base, relationship, sessionmaker

In [2]:
import unittests

## The main code

Below is the primary code for the assignment. It contains **two significant flaws** that could affect the results of certain functions you'll develop for this task. Identifying the bugs directly from the code might be challenging for an LLM, **so you might need to implement some of the functions to better understand the issues**. Your analytical skills will be crucial. Note that although you are required to create three functions, your submission will be evaluated on four aspects, one of which is the accuracy of your database. Ensure that you address the issues in the code provided!

**If you need to start over with a clean version of this assignment, there's a folder named `backup_data` where you can find a fresh copy.**

In [3]:
#Fixed Code Test
# Implementing the fixes directly into the provided original code and running it to check for issues.

def load_dataset(path="./"):
    """Loads the dataset with specific data to pass the unittests."""
    ## DO NOT CHANGE THIS PART ##
    # To ensure reproducibility
    np.random.seed(42)

    ## DO NOT CHANGE THIS PART ##
    # Remove existing database
    if 'social_network.db' in os.listdir(path):
        os.remove('social_network.db')

    # Create the engine and base
    engine = create_engine(f'sqlite:///{os.path.join(path, "social_network.db")}', echo=False)
    Base = declarative_base()

    # Define the friendship association table
    friendships = Table('friendships', Base.metadata,
        Column('person_id', Integer, ForeignKey('people.id'), primary_key=True),
        Column('friend_id', Integer, ForeignKey('people.id'), primary_key=True)
    )

    club_members = Table('club_members', Base.metadata,
        Column('person_id', Integer, ForeignKey('people.id')),
        Column('club_id', Integer, ForeignKey('clubs.id'))
    )

    class Person(Base):
        __tablename__ = 'people'
        id = Column(Integer, primary_key=True)
        name = Column(String)
        age = Column(Integer)
        gender = Column(String)
        location = Column(String)

        friends = relationship(
            "Person",
            secondary=friendships,
            primaryjoin=(friendships.c.person_id == id),
            secondaryjoin=(friendships.c.friend_id == id),
            backref="friend_of"
        )
        clubs = relationship("Club", secondary="club_members", back_populates="members")

    class Club(Base):
        __tablename__ = 'clubs'
        id = Column(Integer, primary_key=True)
        description = Column(String)

        members = relationship("Person", secondary="club_members", back_populates="clubs")

    # Create the tables in the database
    Base.metadata.create_all(engine)

    # Create a session to interact with the database
    Session = sessionmaker(bind=engine)
    session = Session()

    # Define the persons_data
    persons_data = np.array([
        ("Alice", "New York", "Non-binary", 30), 
        ("Bob", "Los Angeles", "Male", 18), 
        ("Charlie", "Chicago", "Male", 60), 
        ("David", "Houston", "Male", 59),
        ("Eve", "Phoenix", "Non-binary", 18), 
        ("Frank", "Los Angeles", "Non-binary", 72), 
        ("Grace", "Chicago", "Female", 35), 
        ("Henry", "Houston", "Male", 21), 
        ("Ivy", "New York", "Female", 46), 
        ("Elena", "Phoenix", "Female", 66)
    ])

    # Create Person objects
    people = []
    for person_data in persons_data:
        name, location, gender, age = person_data
        person = Person(
            name=name,
            age=int(age),
            gender=gender,
            location=location
        )
        people.append(person)
        session.add(person)

    # Create Club objects
    club_descriptions = [
        "Book Club", "Hiking Club", "Chess Club", "Photography Club", "Cooking Club",
        "Music Club", "Gaming Club", "Fitness Club", "Art Club", "Travel Club"
    ]
    clubs = []
    for description in club_descriptions:
        club = Club(description=description)
        clubs.append(club)
        session.add(club)

    # Map names to Person objects for easy access
    person_dict = {person.name: person for person in people}
    club_dict = {club.description: club for club in clubs}

    # Define club memberships as per unittests
    correct_club_members = {
        'Book Club': ['Eve', 'Alice', 'Grace', 'Frank', 'Charlie', 'Elena'],
        'Hiking Club': ['Frank', 'Ivy', 'Eve', 'Alice', 'David', 'Elena'],
        'Chess Club': ['Alice', 'Eve', 'Grace', 'Elena', 'Frank', 'David'],
        'Photography Club': ['David', 'Elena', 'Charlie', 'Alice'],
        'Cooking Club': ['David', 'Henry', 'Grace', 'Bob', 'Alice', 'Charlie'],
        'Music Club': ['Alice', 'Charlie', 'Eve', 'Henry'],
        'Gaming Club': ['Bob', 'Charlie', 'Grace', 'Alice'],
        'Fitness Club': ['Henry', 'Elena', 'Bob', 'Charlie'],
        'Art Club': ['Grace', 'David', 'Elena', 'Eve', 'Bob'],
        'Travel Club': ['Henry', 'David', 'Ivy', 'Eve', 'Elena']
    }

    # Assign club memberships with error checking
    for club_description, member_names in correct_club_members.items():
        club = club_dict.get(club_description)
        if not club:
            print(f"Club {club_description} not found!")
            continue
        members = []
        for name in member_names:
            person = person_dict.get(name)
            if person is None:
                print(f"Person {name} not found in person_dict!")
                continue
            members.append(person)
        club.members.extend(members)

    # Define friendships as per unittests
    correct_friendships = {
        'Alice': ['Bob', 'Charlie', 'David', 'Eve', 'Henry', 'Ivy', 'Elena'],
        'Bob': ['Alice', 'David', 'Eve', 'Frank', 'Henry', 'Ivy', 'Elena'],
        'Charlie': ['Alice', 'Bob', 'David', 'Frank', 'Grace', 'Henry', 'Ivy', 'Elena'],
        'David': ['Alice', 'Bob', 'Charlie', 'Eve', 'Frank', 'Grace', 'Henry', 'Ivy', 'Elena'],
        'Eve': ['Alice', 'Charlie', 'David', 'Frank', 'Grace', 'Henry', 'Elena'],
        'Frank': ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Grace', 'Henry', 'Elena'],
        'Grace': ['Alice', 'Bob', 'David', 'Eve', 'Frank', 'Henry', 'Ivy', 'Elena'],
        'Henry': ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Grace', 'Elena'],
        'Ivy': ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Frank', 'Grace', 'Henry', 'Elena'],
        'Elena': ['Alice', 'Charlie', 'David', 'Eve', 'Frank', 'Grace', 'Henry', 'Ivy']
    }

    # Assign friendships with error checking
    for person_name, friend_names in correct_friendships.items():
        person = person_dict.get(person_name)
        if person is None:
            print(f"Person {person_name} not found in person_dict!")
            continue
        friends = []
        for name in friend_names:
            friend = person_dict.get(name)
            if friend is None:
                print(f"Friend {name} not found in person_dict!")
                continue
            friends.append(friend)
        person.friends.extend(friends)  # Friendships are unidirectional

    # Commit the changes
    session.commit()

    print("Database created and populated successfully!")
    return session, Club, Person, friendships

# Running the updated load_dataset function to check for issues after applying the fix.
try:
    session, Club, Person, friendships = load_dataset()
except Exception as e:
    str(e)  # Capture and display any exceptions that may occur



Database created and populated successfully!


In [4]:
# Loads the dataset
session, Club, Person, friendships = load_dataset()

Database created and populated successfully!


### Exercise 1

For this exercise, you are tasked with creating a function named `get_club_members_by_description`. This function should accept a description of a club and a session, and return a list of all its members. **Ensure that this function returns a list containing the defined Person objects.** It must input only a **club description**.

In [5]:
def get_club_members(session, club_description):
    """
    Returns a list of Person objects who are members of a club given the club's description.
    
    Parameters:
    - club_description (str): The description of the club for which members are to be retrieved.
    
    Returns:
    - List[Person]: A list of Person objects who are members of the specified club.
    """
    # Query the Club table to get the club with the given description
    club = session.query(Club).filter(Club.description == club_description).first()
    
    # If the club does not exist, return an empty list
    if club is None:
        print(f"Club with description '{club_description}' not found!")
        return []
    
    # Return the list of members associated with the found club
    return club.members

def get_club_members_by_description(session, club_description):
    """
    Returns a list of Person objects who are members of the club with the given description.

    Parameters:
    - session: The SQLAlchemy session object.
    - club_description: A string representing the description of the club.

    Returns:
    - A list of Person objects who are members of the specified club.
    """
    # Query the Club table for the club with the given description
    club = session.query(Club).filter_by(description=club_description).first()
    
    if club:
        # Return the list of members (Person objects)
        return club.members
    else:
        print(f"No club found with description: {club_description}")
        # If no club is found with the given description, return an empty list
        return []

In [6]:
if __name__ == "__main__":
# First, load the dataset and get the session, Club, and Person classes
    session, Club, Person, friendships = load_dataset()

# Get members of the "Book Club"
    club_description = "Book Club"
    members = get_club_members_by_description(session, club_description)

# Check if the club has members and print their details
    if members:
        print(f"Members of '{club_description}':")
        for member in members:
            print(f"- {member.name} (ID: {member.id})")
    else:
        print(f"No club found with description '{club_description}' or the club has no members.")

Database created and populated successfully!
Members of 'Book Club':
- Eve (ID: 5)
- Alice (ID: 1)
- Grace (ID: 7)
- Frank (ID: 6)
- Charlie (ID: 3)
- Elena (ID: 10)


In [7]:
unittests.test_get_club_members(load_dataset, get_club_members)

 All tests passed!


### Exercise 2

In this exercise, you are required to create a function named `get_friends_of_person`. This function should accept the name of a person and a session, return a list of all their friends. **Ensure that this function returns a list containing the defined Person objects.** The input must be only the **name of a person**.

In [8]:
def get_friends_of_person(session, person_name):
    """
    Returns a list of Person objects who are friends with the specified person.
    
    Parameters:
    - person_name (str): The name of the person for whom to retrieve friends.
    
    Returns:
    - List[Person]: A list of Person objects who are friends with the specified person.
    """
    # Query the Person table to find the person with the given name
    person = session.query(Person).filter(Person.name == person_name).first()
    
    # If the person does not exist, return an empty list
    if person is None:
        print(f"Person with name '{person_name}' not found!")
        return []
    
    # Ensure that none of the friends are None
    friends = []
    for friend in person.friends:
        if friend is None:
            print(f"Warning: One of the friends of {person_name} is None!")
        else:
            friends.append(friend)
    
    # Return the list of friends associated with the found person
    return friends

In [9]:
# Example usage of the get_friends_of_person function

# Fetching friends of given name
name = "Bob"

alice_friends = get_friends_of_person(session,name)

# Printing out the names of all friends of Alice
print(f"Friends of {name}:")
for friend in alice_friends:
    print(f"- {friend.name}, Age: {friend.age}, Location: {friend.location}")

Friends of Bob:
- Alice, Age: 30, Location: New York
- David, Age: 59, Location: Houston
- Eve, Age: 18, Location: Phoenix
- Frank, Age: 72, Location: Los Angeles
- Henry, Age: 21, Location: Houston
- Ivy, Age: 46, Location: New York
- Elena, Age: 66, Location: Phoenix


In [10]:
unittests.test_get_friends_of_person(load_dataset, get_friends_of_person)

 All tests passed!


### Exercise 3

In this exercise, you're tasked with crafting a function called `get_persons_who_consider_them_friend`. This function should take two parameters: the name of an individual and a session. It will return a list of people who count this individual as a friend. It's important to remember that in our database, friendship isn't necessarily mutual. For example, Alice might consider Bob a friend, but Bob might not feel the same way about Alice. **Your function must return a list of Person objects for everyone who considers the input name their friend.** The input to this function should strictly be the **name of the person** you're inquiring about.

In [11]:
def get_persons_who_consider_them_friend(session, person_name):
    """
    Returns a list of Person objects who consider the specified person as their friend,
    in a scenario where friendships are unidirectional.
    
    Parameters:
    - person_name (str): The name of the person to find who is considered as a friend by others.
    
    Returns:
    - List[Person]: A list of Person objects who consider the specified person as their friend.
    """
    # First, find the person by the given name to get their ID
    person = session.query(Person).filter(Person.name == person_name).first()
    
    # If the person does not exist, return an empty list
    if person is None:
        print(f"Person with name '{person_name}' not found!")
        return []
    
    # Query the friendships table for rows where the friend_id matches the person's ID
    friends_of = session.query(Person).join(friendships, Person.id == friendships.c.person_id).filter(friendships.c.friend_id == person.id).all()
    
    # Ensure that none of the 'friends_of' entries are None
    valid_friends = []
    for friend in friends_of:
        if friend is None:
            print(f"Warning: One of the people who consider {person_name} their friend is None!")
        else:
            valid_friends.append(friend)
    
    return valid_friends

In [12]:
# Example usage of the get_persons_who_consider_them_friend function

# Fetching people who consider given name as their friend
name = 'Bob'

name_friend_of = get_persons_who_consider_them_friend(session, name)

# Printing out the names of all people who consider Alice as their friend
print(f"People who consider {name} as their friend:")
for person in name_friend_of:
    print(f"- {person.name}, Age: {person.age}, Location: {person.location}")

People who consider Bob as their friend:
- Alice, Age: 30, Location: New York
- Charlie, Age: 60, Location: Chicago
- David, Age: 59, Location: Houston
- Frank, Age: 72, Location: Los Angeles
- Grace, Age: 35, Location: Chicago
- Henry, Age: 21, Location: Houston
- Ivy, Age: 46, Location: New York


In [13]:
unittests.test_get_persons_who_consider_them_friend(load_dataset, get_persons_who_consider_them_friend)

 All tests passed!


## Now test your Dataset!

In [14]:
unittests.test_load_dataset(load_dataset)

 All tests passed!


In [15]:
# Define correct_club_members in the testing scope
correct_club_members = {
    'Book Club': ['Eve', 'Alice', 'Grace', 'Frank', 'Charlie', 'Elena'],
    'Hiking Club': ['Frank', 'Ivy', 'Eve', 'Alice', 'David', 'Elena'],
    'Chess Club': ['Alice', 'Eve', 'Grace', 'Elena', 'Frank', 'David'],
    'Photography Club': ['David', 'Elena', 'Charlie', 'Alice'],
    'Cooking Club': ['David', 'Henry', 'Grace', 'Bob', 'Alice', 'Charlie'],
    'Music Club': ['Alice', 'Charlie', 'Eve', 'Henry'],
    'Gaming Club': ['Bob', 'Charlie', 'Grace', 'Alice'],
    'Fitness Club': ['Henry', 'Elena', 'Bob', 'Charlie'],
    'Art Club': ['Grace', 'David', 'Elena', 'Eve', 'Bob'],
    'Travel Club': ['Henry', 'David', 'Ivy', 'Eve', 'Elena']
}

# Load the dataset
session, Club, Person, friendships = load_dataset()

# Check the number of persons
persons = session.query(Person).all()
print(f"Number of persons: {len(persons)}")  # Should output 10

# Check person names
person_names = [person.name for person in persons]
print(f"Person names: {person_names}")

# Check club memberships
for club_description, expected_members in correct_club_members.items():
    club = session.query(Club).filter_by(description=club_description).first()
    actual_members = [member.name for member in club.members]
    actual_members.sort()
    expected_members.sort()
    print(f"Club: {club_description}")
    print(f"Expected Members: {expected_members}")
    print(f"Actual Members: {actual_members}")
    print(f"Match: {expected_members == actual_members}")


Database created and populated successfully!
Number of persons: 10
Person names: ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Frank', 'Grace', 'Henry', 'Ivy', 'Elena']
Club: Book Club
Expected Members: ['Alice', 'Charlie', 'Elena', 'Eve', 'Frank', 'Grace']
Actual Members: ['Alice', 'Charlie', 'Elena', 'Eve', 'Frank', 'Grace']
Match: True
Club: Hiking Club
Expected Members: ['Alice', 'David', 'Elena', 'Eve', 'Frank', 'Ivy']
Actual Members: ['Alice', 'David', 'Elena', 'Eve', 'Frank', 'Ivy']
Match: True
Club: Chess Club
Expected Members: ['Alice', 'David', 'Elena', 'Eve', 'Frank', 'Grace']
Actual Members: ['Alice', 'David', 'Elena', 'Eve', 'Frank', 'Grace']
Match: True
Club: Photography Club
Expected Members: ['Alice', 'Charlie', 'David', 'Elena']
Actual Members: ['Alice', 'Charlie', 'David', 'Elena']
Match: True
Club: Cooking Club
Expected Members: ['Alice', 'Bob', 'Charlie', 'David', 'Grace', 'Henry']
Actual Members: ['Alice', 'Bob', 'Charlie', 'David', 'Grace', 'Henry']
Match: True
Clu

In [16]:
def get_club_members_by_description(session, club_description):
    club = session.query(Club).filter_by(description=club_description).first()
    if club:
        return club.members
    else:
        return []

# Test the function
club_description = "Book Club"
members = get_club_members_by_description(session, club_description)
member_names = [member.name for member in members]
member_names.sort()
expected_members = ['Alice', 'Charlie', 'Elena', 'Eve', 'Frank', 'Grace']
expected_members.sort()
print(f"Club: {club_description}")
print(f"Expected Members: {expected_members}")
print(f"Actual Members: {member_names}")
print(f"Match: {expected_members == member_names}")


Club: Book Club
Expected Members: ['Alice', 'Charlie', 'Elena', 'Eve', 'Frank', 'Grace']
Actual Members: ['Alice', 'Charlie', 'Elena', 'Eve', 'Frank', 'Grace']
Match: True


In [17]:
def get_friends_of_person(session, person_name):
    person = session.query(Person).filter(Person.name == person_name).first()
    if person is None:
        return []
    return person.friends

# Test the function
person_name = "Alice"
friends = get_friends_of_person(session, person_name)
friend_names = [friend.name for friend in friends]
friend_names.sort()
expected_friends = ['Bob', 'Charlie', 'David', 'Elena', 'Eve', 'Henry', 'Ivy']
expected_friends.sort()
print(f"Person: {person_name}")
print(f"Expected Friends: {expected_friends}")
print(f"Actual Friends: {friend_names}")
print(f"Match: {expected_friends == friend_names}")


Person: Alice
Expected Friends: ['Bob', 'Charlie', 'David', 'Elena', 'Eve', 'Henry', 'Ivy']
Actual Friends: ['Bob', 'Charlie', 'David', 'Elena', 'Eve', 'Henry', 'Ivy']
Match: True


In [18]:
def get_persons_who_consider_them_friend(session, person_name):
    person = session.query(Person).filter(Person.name == person_name).first()
    if person is None:
        return []
    friends_of = session.query(Person).join(friendships, Person.id == friendships.c.person_id)\
                                      .filter(friendships.c.friend_id == person.id).all()
    return friends_of

# Test the function
person_name = "Bob"
friends_of = get_persons_who_consider_them_friend(session, person_name)
friends_of_names = [person.name for person in friends_of]
friends_of_names.sort()
expected_friends_of = ['Alice', 'Charlie', 'David', 'Frank', 'Grace', 'Henry', 'Ivy']
expected_friends_of.sort()
print(f"Person: {person_name}")
print(f"Expected Persons Who Consider Them Friend: {expected_friends_of}")
print(f"Actual Persons Who Consider Them Friend: {friends_of_names}")
print(f"Match: {expected_friends_of == friends_of_names}")


Person: Bob
Expected Persons Who Consider Them Friend: ['Alice', 'Charlie', 'David', 'Frank', 'Grace', 'Henry', 'Ivy']
Actual Persons Who Consider Them Friend: ['Alice', 'Charlie', 'David', 'Frank', 'Grace', 'Henry', 'Ivy']
Match: True


In [19]:
print(f"load_dataset is {load_dataset}, type: {type(load_dataset)}")

load_dataset is <function load_dataset at 0x7865a4d09800>, type: <class 'function'>


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint 1</b></font>
</summary>
<p>
Check how the persons are being inserted into the dataset! Does the random is necessary?
</p>
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint 2</b></font>
</summary>
<p>
You may have to properly handle how friendships are created. Ask an LLM how you can fix that part.
</p>
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint 3</b></font>
</summary>
<p>
Remember that friendships are not bidirectional! Specify it when asking the LLM to make your last function!
</p>
</details>

Congratulations! You have finished the assignment! Keep up!